# Perceptron (Online) Polynomial Regression

We will describe a procedure in which one can perform online polynomial
regression without exploding the values into polynomial features
(e.g. using `sklearn`'s `PolynomialFeatures`).
This should lessen the memory burden of the polynomial regression,
notably when fitting very high order polynomials.

### Neural Network as an Equation

The implementation of a fully connected neural network is achieved
through the matrix multiplication of the input vector with a matrix
of weights for a network layer.  The bias terms are added after
the multiplication and the activation function is performed on the
result.  The procedure is repeated for every layer in the network.

For example, a neural network with two input nodes (annotated as $i$),
three nodes in the hidden layer ($h$) and two output nodes ($o$);
will have the following matrices and bias terms:

$$
L_1 =
\begin{bmatrix}
    w_{hi11} & w_{hi12} \\
    w_{hi21} & w_{hi22} \\
    w_{hi31} & w_{hi32} \\
\end{bmatrix}
L_{1bias} =
\begin{bmatrix}
    w_{h1} \\
    w_{h2} \\
    w_{h3} \\
\end{bmatrix}
\\
L_2 =
\begin{bmatrix}
    w_{oh11} & w_{oh12} & w_{oh13} \\
    w_{oh21} & w_{oh22} & w_{oh23} \\
\end{bmatrix}
L_{2bias} =
\begin{bmatrix}
    w_{o1} \\
    w_{o2} \\
\end{bmatrix}
$$

Where $L_1$ holds the weights between the input layer and the hidden layer,
$L_{1bias}$ are the bias terms added to the hidden layer,
$L_2$ the weights between the hidden and output layers,
and $_{2bias}$ the bias terms added to the output layer.
The indexes on the weights summarize with connection the weight apply to.
For example, $w_{hi32}$ means the weight on the connection between the 2nd
node in the input layer (the $2$ is in the same position as the $i$in the subscript)
and the 3rd node in the hidden layer.  The bias terms have only one
subscript since they are applied to a single layer only.

In order to execute a forward pass through the described network
we right multiply the input vector (call it $\vec{x}$) with the
layer matrix, add the bias and pass it through the activation
function (call it $f$).  For this network the forward pass is then:

$$
\vec{x} =
\begin{bmatrix}
    x_{0} \\
    x_{1} \\
\end{bmatrix}
\\
\hat{y} =
f\left(
\begin{bmatrix}
    w_{oh11} & w_{oh12} & w_{oh13} \\
    w_{oh21} & w_{oh22} & w_{oh23} \\
\end{bmatrix}
f\left(
\begin{bmatrix}
    w_{hi11} & w_{hi12} \\
    w_{hi21} & w_{hi22} \\
    w_{hi31} & w_{hi32} \\
\end{bmatrix}
\begin{bmatrix}
    x_{0} \\
    x_{1} \\
\end{bmatrix}
+
\begin{bmatrix}
    w_{h1} \\
    w_{h2} \\
    w_{h3} \\
\end{bmatrix}
\right)
+
\begin{bmatrix}
    w_{o1} \\
    w_{o2} \\
\end{bmatrix}
\right)
$$

We can reduce this equation to a simpler form if we substitute
the matrix names we defined above:

$$
\hat{y} = f(L_2 f(L_1 \vec{x} + L_{1bias}) + L_{2bias})
$$

OK, this is simple enough to reason about.
In the same fashion, we can now add more layer to our network.
For example, with three layers:

$$
\hat{y} = f(L_3 f(L_2 f(L_1 \vec{x} + L_{1bias}) + L_{2bias}) + L_{3bias})
$$

More layers follow by left multiplying the matrix,
adding the bias and applying the activation function of the layer.

### Polynomial Division

In college we learn a procedure to divide one polynomial by another.
The procedure consists of adding multiples of the divisor to the dividend
in order to remove the highest order term of the dividend.
Once the dividend's order is smaller than the divisor the procedure stops,
what remains from the dividend is the rest and the sum of the multiples
of the divisor is the quotient.

We are only interested in a subset of this procedure,
namely the division of a polynomial by a polynomial in the form $(x - r)$.
If the remainder of a division of a polynomial by a polynomial
in the form $(x - r)$ is zero, this means that $r$ is one of the real
roots of the divided polynomial.
This also means that $(x - r)$ is one of the factors of the polynomial,
i.e. terms to multiply together to reach the polynomial.

For example, given the following polynomial:

$$
x^3 + 2x^2 - 5x - 6
$$

We can divide it by $(x + 1)$, $(x - 2)$ and $(x + 3)$; since the roots
of this polynomial are $-3$, $-1$ and $2$.
First let's prove to ourselves that these are the roots of the polynomial:

In [3]:
def f(x):
    return x**3 + 2*x**2 - 5*x - 6

print(f(-3))
print(f(-1))
print(f(2))

0
0
0


Now we will divide

$$
\begin{array}{r|cccc}
        & - x^2 & -  x   & + 6  &     \\
\hline
(x + 1) &   x^3 & + 2x^2 & - 5x & - 6 \\ 
      + & - x^3 & -  x^2 &      &     \\ 
\hline
        &       &    x^2 & - 5x & - 6 \\ 
      + &       & -  x^2 & -  x &     \\ 
\hline
        &       &        & - 6x & - 6 \\ 
      + &       &        &   6x & + 6 \\ 
\hline
        &       &        &      &   0 \\ 
\end{array}
$$

$$
\begin{array}{r|cccc}
    &   1 &   2         & - 5 & - 6 \\ 
\hline
- 1 &   1 & (2 + 1(-1))  & (-5 + (-1)(-6)) & (-6 + (-1)(-6)) \\ 
\end{array}
$$

$$
\begin{array}{r|cccc}
    &   1 &   2 & - 5 & - 6 \\ 
\hline
    & (1 + (0)(-1)) & (2 + (1)(-1))  & (-5 + (1)(-1)) & (-6 + (-6)(-1)) \\ 
- 1 &             1 &              1 &            - 6 &               0 \\ 
\end{array}
$$

Now we generalize the syntetic division.

$$
w_nx^n + \cdots + w_3x^3 + w_2x^2 + w_1x^1 + w_0x^0
$$

The syntetic division will look like:

$$
\begin{array}{r|cccccc}
  &          w_3 &            w_2 & w_1 & w_0 \\ 
\hline
  & (w_3 + y_4 x) & (w_2 + y_3 x) & (w_1 + y_2 x) & (w_0 + y_1 x) \\ 
x &           y_3 &           y_2 &           y_1 &           y_0 \\ 
\end{array}
$$

$$
\begin{align}
w_4 &= 0 \\
y_5 &= 0 \\
y_4 &= w_4 + y_5 x = 0 \\
y_3 &= w_3 + y_4 x = w_3 \\
y_2 &= w_2 + y_3 x \\
y_1 &= w_1 + y_2 x \\
y_0 &= w_0 + y_1 x \\
\end{align}
$$

$$
\begin{align}
y_0 &= w_0 + (w_1 + y_2 x) x \\
    &= w_0 + (w_1 + (w_2 + y_3 x) x) x \\
    &= w_0 + (w_1 + (w_2 + w_3 x) x) x \\
    &= w_0 + (w_1 + w_2 x + w_3 x^2) x \\
    &= w_0 + w_1 x + w_2 x^2 + w_3 x^3 = y \\
\end{align}
$$

### Comparing

$$
\begin{align}
y &= w_0 + (w_1 + (w_2 + w_3 x) x) x \\
y &= x (x (w_3 x + w_2) + w_1) + w_0 \\
\end{align}
$$

And

$$
\hat{y} = f(L_3 f(L_2 f(L_1 \vec{x} + L_{1bias}) + L_{2bias}) + L_{3bias})
$$

$$
\begin{align}
\hat{y} &= L_3 (L_2 (x (0 + L_{0bias}) + L_{1bias}) + L_{2bias}) + L_{3bias} \\
\hat{y} &= x (x (x (0 + L_{0bias}) + L_{1bias}) + L_{2bias}) + L_{3bias} \\
y       &= x (x (x (0 + w_3) + w_2) + w_1) + w_0 \\
\end{align}
$$

In summary, a neural network with:

- Single node per layer
- Input zero
- Bias on on input layer
- All weights between nodes set to $x$
- All activation functions as identity

Performs the evaluation of a polynomial.

Moreover, we can optimize the bias terms at each layer.
For this we need a loss function, say:

$$
E = (y - \hat{y})^2
$$

All gradients can be easily precomputed, since:

$$
\frac{\partial{E}}{\partial{\hat{y}}} = 2(\hat{y} - y)
$$

We have

$$
\begin{align}
\frac{\partial{E}}{\partial{L_{3bias}}} &= 2(\hat{y} - y) \\
\frac{\partial{E}}{\partial{L_{2bias}}} &= 2x(\hat{y} - y) \\
\frac{\partial{E}}{\partial{L_{1bias}}} &= 2x^2(\hat{y} - y) \\
\frac{\partial{E}}{\partial{L_{0bias}}} &= 2x^3(\hat{y} - y) \\
\end{align}
$$

Note that we have been using $x$ and $y$ as scalars,
yet nothing prevents us from using min-batch vectors instead.
The difference in that case is that we need to change the loss
function and derivatives as follows:

$$
N = \tt{number\ of\ items\ in\ mini-batch} \\
E = \tt{mean} \left( \left( y - \hat{y} \right)^2 \right) \\
\frac{\partial{E}}{\partial{\hat{y}}} = \frac{2(\hat{y} - y)}{N} \\
$$